In [0]:
# Load the Drive helper and mount

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls gdrive/'My Drive'/EE219/project5/tweet_data


Project5_Readme.txt	tweets_#nfl.txt       tweets_#superbowl.txt
tweets_#gohawks.txt	tweets_#patriots.txt
tweets_#gopatriots.txt	tweets_#sb49.txt


In [0]:
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import json
import datetime


import datetime, time
import pytz
import collections
X_data = [] 
Y_data = []

In [0]:
##Beofre 8AM
hashtags = ['#gohawks', '#gopatriots', '#nfl', '#patriots', '#sb49', '#superbowl']
#hashtags = ['#gohawks']
my_list = []
X = {}
num_tweets = {}
num_followers = 0
num_retweets = 0
max_followers = {}
total_tweets = 0
total_retweets = {}
sum_followers = {}
regression_result = []
user_mentions = {}
impressions_count = {}
author_names = {}

url_count = {}
fav_count = {}
count = 0
pst_tz = pytz.timezone('America/Los_Angeles')
X.clear()
X_data.clear()
Y_data.clear()
for h in hashtags:
  print("+++++++++++++++++++++++++++++++++")
  print("### Working on hashtag ",h)
  print("+++++++++++++++++++++++++++++++++")   
  with open('gdrive/My Drive/EE219/project5/tweet_data/tweets_' + h + '.txt', 'r', encoding="utf8") as file:
      for line in file:
          parsed_obj = json.loads(line)
          #print(parsed_obj)
          citation_time = parsed_obj['citation_date']
            #creating a copy 
          citation_time_hr = citation_time 
          citation_time_hr -= citation_time_hr % 3600
          time_inst_hr = citation_time_hr
          
         # time_inst = datetime.datetime.fromtimestamp(unix_time, pst_tz).strftime('%Y-%m-%d %H:%M:%S')
          #time_inst_hr = datetime.datetime.fromtimestamp(citation_time, pst_tz).strftime('%H')
          
          # Time given in the problem is in PST. Dont know how to invoke the pst format to get the unix_time. So adding +8 into it. 
          feb1st8am_time = datetime.datetime(2015, 2, 1, 16, 0).timestamp()
          #feb1st8pm_time = datetime.datetime(2015, 2, 2, 4, 0).timestamp()
        #  X['time_inst_hr'].append()
         # print(feb1st8am_time)
         # print(citation_time)
       
          if citation_time < feb1st8am_time: 
              #print("working on citation_time=",citation_time)
              first_post = parsed_obj['firstpost_date']
              count += 1
              num_followers = parsed_obj['author']['followers']
              author_name = parsed_obj['author']['name']
              num_retweets = parsed_obj['metrics']['citations']['total']
              total_tweets = 1 
              mentions = len(parsed_obj['tweet']['entities']['user_mentions'])
              urls = len(parsed_obj['tweet']['entities']['urls'])
              favorite_count = parsed_obj['tweet']['favorite_count']
              impressions_num = parsed_obj['metrics']['impressions']
              author_name = parsed_obj['author']['name']

              if(time_inst_hr in num_tweets ):
                  num_tweets[time_inst_hr] += total_tweets
              else:
                 num_tweets[time_inst_hr] = total_tweets  

              if(time_inst_hr in total_retweets ):
                  total_retweets[time_inst_hr] += num_retweets
              else:
                 total_retweets[time_inst_hr] = num_retweets 

              if(time_inst_hr in sum_followers):
                  sum_followers[time_inst_hr] += num_followers
              else:
                 sum_followers[time_inst_hr] = num_followers     

              if(time_inst_hr in max_followers):
                if num_followers > max_followers[time_inst_hr]:
                  max_followers[time_inst_hr] = num_followers
              else:
                 max_followers[time_inst_hr] = num_followers  

              if(time_inst_hr in user_mentions):
                  user_mentions[time_inst_hr] += mentions
              else:
                 user_mentions[time_inst_hr] = mentions

              if(time_inst_hr in url_count):
                  url_count[time_inst_hr] += urls
              else:
                 url_count[time_inst_hr] = urls   

              if(time_inst_hr in fav_count):
                  fav_count[time_inst_hr] += favorite_count
              else:
                 fav_count[time_inst_hr] = favorite_count   

              if(time_inst_hr in impressions_count):
                  impressions_count[time_inst_hr] += impressions_num
              else:
                 impressions_count[time_inst_hr] = impressions_num 

              if(time_inst_hr in author_names):
                  author_names[time_inst_hr].append(author_name)
              else:
                 author_names[time_inst_hr] = [author_name]    
          
              # reset the values to 0 for next iteration
              num_followers = 0
              num_retweets  = 0
              total_tweets  = 0
          else: 
              #print("skipped citation_time",citation_time)
              num_followers = 0
              num_retweets  = 0
              total_tweets  = 0
            
          #if count == 100:
          #  break
# print(author_names)
X_data = [] 
Y_data = []
#just to get the keys    
for key in sorted (num_tweets.keys()):
    #print(key)
    #value = int(key)
    time_of_day = int(int(datetime.datetime.fromtimestamp(key).strftime('%H')))
    day_value = int(datetime.datetime.fromtimestamp(key).strftime('%w'))
    #authors_list = author_names[key]
    unique_authors_count = len(set(author_names[key]))
    #print("authrs list and count:",authors_list,unique_authors_count)
    X[key] = [num_tweets[key],total_retweets[key],sum_followers[key],max_followers[key],user_mentions[key],url_count[key],fav_count[key],impressions_count[key],unique_authors_count]
    # Num_tweets, total_retweets, sum of followers , max_followers , time of tweet , user mentions, url_count,fav_count,impressions, unique_authors_count
    X_data.append([num_tweets[key],total_retweets[key],sum_followers[key],max_followers[key],time_of_day,user_mentions[key],url_count[key],fav_count[key],impressions_count[key],unique_authors_count,day_value])
    Y_data.append([num_tweets[key]])
#print(X_data)
#print(Y_data)
Y = collections.deque(Y_data)
#print(Y)
Y.rotate(-1)
#print(Y)
Y = list(Y)
#print(Y)


Y = np.ravel(Y)

+++++++++++++++++++++++++++++++++
### Working on hashtag  #gohawks
+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++
### Working on hashtag  #gopatriots
+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++
### Working on hashtag  #nfl
+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++
### Working on hashtag  #patriots
+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++
### Working on hashtag  #sb49
+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++
### Working on hashtag  #superbowl
+++++++++++++++++++++++++++++++++


In [0]:
from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
'max_depth': [10, 20, 40, 60, 80, 100, 200, None],
'max_features': ['auto', 'sqrt'],
'min_samples_leaf': [1, 2, 4],
'min_samples_split': [2, 5, 10],
'n_estimators': [200, 400, 600, 800, 1000,
1200, 1400, 1600, 1800, 2000]
}
# Create a based model

from sklearn.ensemble import GradientBoostingRegressor

gb  = GradientBoostingRegressor(random_state=0)


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = gb, param_grid = param_grid, 
                         cv = KFold(5, shuffle=True), scoring='neg_mean_squared_error')

grid_search.fit(X_data, Y)
grid_search.best_params_

{'max_depth': 60,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 400}

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size = 0.2, random_state = 0)

#def evaluate(model, test_features, test_labels):
    #predictions = model.predict(test_features)
    #errors = abs(predictions - test_labels)
    #mape = 100 * np.mean(errors / test_labels)
    #accuracy = 100 - mape
    #print('Model Performance')
    #print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    #return accuracy

#from sklearn.metrics import accuracy_score
best_grid = grid_search.best_estimator_
print(best_grid)
#grid_accuracy = evaluate(best_grid, X_test, y_test)


gb =  GradientBoostingRegressor(random_state=0, n_estimators=400, max_depth=60, max_features='sqrt', 
                                  min_samples_leaf=2, min_samples_split=5)
gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)

from sklearn import metrics

print("Mean Square Error", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean-Square Error" , np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# print("Accuracy ", metrics.accuracy_score(Y, Y_pred))

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=60,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=2, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=400,
                          n_iter_no_change=None, presort='auto', random_state=0,
                          subsample=1.0, tol=0.0001, validation_fraction=0.1,
                          verbose=0, warm_start=False)


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Square Error 60095.710113543275
Root Mean-Square Error 245.14426388056336


In [0]:
#During Superbowl

hashtags = ['#gohawks', '#gopatriots', '#nfl', '#patriots', '#sb49', '#superbowl']
#hashtags = ['#gohawks']
my_list = []
X = {}
num_tweets = {}
num_followers = 0
num_retweets = 0
max_followers = {}
total_tweets = 0
total_retweets = {}
sum_followers = {}
regression_result = []
user_mentions = {}
impressions_count = {}
author_names = {}

url_count = {}
fav_count = {}
count = 0
pst_tz = pytz.timezone('America/Los_Angeles')
X.clear()
X_data.clear()
Y_data.clear()
for h in hashtags:
  print("+++++++++++++++++++++++++++++++++")
  print("### Working on hashtag ",h)
  print("+++++++++++++++++++++++++++++++++")   
  with open('gdrive/My Drive/EE219/project5/tweet_data/tweets_' + h + '.txt', 'r', encoding="utf8") as file:
      for line in file:
          parsed_obj = json.loads(line)
          #print(parsed_obj)
          citation_time = parsed_obj['citation_date']
          time_inst = datetime.datetime.fromtimestamp(citation_time, pst_tz).strftime('%Y-%m-%d %H:%M:%S')
          time_inst_hr = datetime.datetime.fromtimestamp(citation_time, pst_tz).strftime('%H')
          #Time Zone given in the problem are in PST format. So added +8 below 2 lines as I dont know how to pick the PST time here. 
          feb1st8pm_time = datetime.datetime(2015, 2, 2, 4, 0).timestamp()
          feb1st8am_time = datetime.datetime(2015, 2, 1, 16, 0).timestamp()
          
          #print("feb1st",feb1st8am_time)
          #print("citation",citation_time)
          #print("time_inst", time_inst)
         
          
          #print("citation_rounded",citation_time  )
          #time_inst = datetime.datetime.fromtimestamp(citation_time, pst_tz).strftime('%Y-%m-%d %H:%M:%S')
          #print("time_inst", time_inst)
          #feb1st8am_time = feb1st8am_time + 300
          keys = np.arange(feb1st8am_time,feb1st8pm_time,300)
          #print(keys)
          #print(len(keys))
          time_inst_8am = datetime.datetime.fromtimestamp(feb1st8am_time, pst_tz).strftime('%Y-%m-%d %H:%M:%S')
          time_inst_8pm = datetime.datetime.fromtimestamp(feb1st8pm_time, pst_tz).strftime('%Y-%m-%d %H:%M:%S')
          #print("8am =",time_inst_8am)
          #print("8pm =",time_inst_8pm)
        #  X['time_inst_hr'].append()
         # print(feb1st8am_time)
         # print(citation_time)
          count += 1
          if  citation_time >= feb1st8am_time and citation_time < feb1st8pm_time:
              #print("working on citation_time=",citation_time)
              time_inst = datetime.datetime.fromtimestamp(citation_time, pst_tz).strftime('%Y-%m-%d %H:%M:%S')
              #print("time_inst", time_inst)
              first_post = parsed_obj['firstpost_date']
              # Round down to  5 minute value
              citation_time -= citation_time % 300
              # Key value is the rounded citation time. 
              time_inst_5m = citation_time
              num_followers = parsed_obj['author']['followers']
              author_name = parsed_obj['author']['name']
              num_retweets = parsed_obj['metrics']['citations']['total']
              total_tweets = 1 
              mentions = len(parsed_obj['tweet']['entities']['user_mentions'])
              urls = len(parsed_obj['tweet']['entities']['urls'])
              favorite_count = parsed_obj['tweet']['favorite_count']
              impressions_num = parsed_obj['metrics']['impressions']
              author_name = parsed_obj['author']['name']
              if(time_inst_5m in num_tweets ):
                  num_tweets[time_inst_5m] += total_tweets
              else:
                 num_tweets[time_inst_5m] = total_tweets  

              if(time_inst_5m in total_retweets ):
                  total_retweets[time_inst_5m] += num_retweets
              else:
                 total_retweets[time_inst_5m] = num_retweets 

              if(time_inst_hr in sum_followers):
                  sum_followers[time_inst_5m] += num_followers
              else:
                 sum_followers[time_inst_5m] = num_followers     

              if(time_inst_5m in max_followers):
                if num_followers > max_followers[time_inst_5m]:
                  max_followers[time_inst_5m] = num_followers
              else:
                 max_followers[time_inst_5m] = num_followers  

              if(time_inst_5m in user_mentions):
                  user_mentions[time_inst_5m] += mentions
              else:
                 user_mentions[time_inst_5m] = mentions

              if(time_inst_5m in url_count):
                  url_count[time_inst_5m] += urls
              else:
                 url_count[time_inst_5m] = urls   

              if(time_inst_5m in fav_count):
                  fav_count[time_inst_5m] += favorite_count
              else:
                 fav_count[time_inst_5m] = favorite_count   

              if(time_inst_5m in impressions_count):
                  impressions_count[time_inst_5m] += impressions_num
              else:
                 impressions_count[time_inst_5m] = impressions_num 

              if(time_inst_5m in author_names):
                  author_names[time_inst_5m].append(author_name)
              else:
                 author_names[time_inst_5m] = [author_name]    
          
              # reset the values to 0 for next iteration
              num_followers = 0
              num_retweets  = 0
              total_tweets  = 0
          else: 
              #print("skipped citation_time",citation_time)
              num_followers = 0
              num_retweets  = 0
              total_tweets  = 0
          #   print (count)
          #if count == 5:
          #    break
          
      #print(author_names)
    #  X_data = [] 
    #  Y_data = []
      #just to get the keys  
    
for key in sorted (num_tweets.keys()):
    #print(key)
    #value = int(key)
    time_of_day = int(int(datetime.datetime.fromtimestamp(key).strftime('%H')))
    #authors_list = author_names[key]
    unique_authors_count = len(set(author_names[key]))
    day_value = int(datetime.datetime.fromtimestamp(key).strftime('%w'))
    #print("authrs list and count:",authors_list,unique_authors_count)
    X[key] = [num_tweets[key],total_retweets[key],sum_followers[key],max_followers[key],user_mentions[key],url_count[key],fav_count[key],impressions_count[key],unique_authors_count]
    # Num_tweets, total_retweets, sum of followers , max_followers , time of tweet , user mentions, url_count,fav_count,impressions, unique_authors_count
    X_data.append([num_tweets[key],total_retweets[key],sum_followers[key],max_followers[key],time_of_day,user_mentions[key],url_count[key],fav_count[key],impressions_count[key],unique_authors_count,day_value])
    Y_data.append([num_tweets[key]])
    
#print(X_data)
#print(Y_data, len(Y_data))
Y = collections.deque(Y_data)
#print(Y)
Y.rotate(-1)
# print(Y)
Y = list(Y)
#print(Y)
      

Y = np.ravel(Y)

+++++++++++++++++++++++++++++++++
### Working on hashtag  #gohawks
+++++++++++++++++++++++++++++++++


In [0]:
from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
'max_depth': [10, 20, 40, 60, 80, 100, 200, None],
'max_features': ['auto', 'sqrt'],
'min_samples_leaf': [1, 2, 4],
'min_samples_split': [2, 5, 10],
'n_estimators': [200, 400, 600, 800, 1000,
1200, 1400, 1600, 1800, 2000]
}
# Create a based model
from sklearn.ensemble import GradientBoostingRegressor

gb  = GradientBoostingRegressor(random_state=0)


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = gb, param_grid = param_grid, 
                         cv = KFold(5, shuffle=True), scoring='neg_mean_squared_error')

grid_search.fit(X_data, Y)
grid_search.best_params_





In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size = 0.2, random_state = 0)

#def evaluate(model, test_features, test_labels):
    #predictions = model.predict(test_features)
    #errors = abs(predictions - test_labels)
    #mape = 100 * np.mean(errors / test_labels)
    #accuracy = 100 - mape
    #print('Model Performance')
    #print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    #return accuracy

#from sklearn.metrics import accuracy_score
best_grid = grid_search.best_estimator_
print(best_grid)
#grid_accuracy = evaluate(best_grid, X_test, y_test)

gb =  GradientBoostingRegressor(random_state=0, n_estimators=200, max_depth=20, max_features='sqrt', 
                                  min_samples_leaf=1, min_samples_split=2)
gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)

from sklearn import metrics

print("Mean Square Error", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean-Square Error" , np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# print("Accuracy ", metrics.accuracy_score(Y, Y_pred))

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=20,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=200,
                          n_iter_no_change=None, presort='auto', random_state=0,
                          subsample=1.0, tol=0.0001, validation_fraction=0.1,
                          verbose=0, warm_start=False)
Mean Square Error 1322530.1697559783
Root Mean-Square Error 1150.0131172103988


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
hashtags = ['#gohawks', '#gopatriots', '#nfl', '#patriots', '#sb49', '#superbowl']
#hashtags = ['#gohawks']
my_list = []
X = {}
num_tweets = {}
num_followers = 0
num_retweets = 0
max_followers = {}
total_tweets = 0
total_retweets = {}
sum_followers = {}
regression_result = []
user_mentions = {}
impressions_count = {}
author_names = {}

url_count = {}
fav_count = {}
count = 0
pst_tz = pytz.timezone('America/Los_Angeles')
X.clear()
X_data.clear()
Y_data.clear()
for h in hashtags:
  print("+++++++++++++++++++++++++++++++++")
  print("### Working on hashtag ",h)
  print("+++++++++++++++++++++++++++++++++")   
  with open('gdrive/My Drive/EE219/project5/tweet_data/tweets_' + h + '.txt', 'r', encoding="utf8") as file:
      for line in file:
          parsed_obj = json.loads(line)
          #print(parsed_obj)
          citation_time = parsed_obj['citation_date']
               #creating a copy 
          citation_time_hr = citation_time 
          citation_time_hr -= citation_time_hr % 3600
          time_inst_hr = citation_time_hr
         # time_inst = datetime.datetime.fromtimestamp(unix_time, pst_tz).strftime('%Y-%m-%d %H:%M:%S')
          #time_inst_hr = datetime.datetime.fromtimestamp(citation_time, pst_tz).strftime('%H')
          
          # Time given in the problem is in PST. Dont know how to invoke the pst format to get the unix_time. So adding +8 into it. 
          #feb1st8am_time = datetime.datetime(2015, 2, 1, 16, 0).timestamp()
          feb1st8pm_time = datetime.datetime(2015, 2, 2, 4, 0).timestamp()
        #  X['time_inst_hr'].append()
         # print(feb1st8am_time)
         # print(citation_time)
       
          if citation_time >= feb1st8pm_time: 
              #print("working on citation_time=",citation_time)
              first_post = parsed_obj['firstpost_date']
              count += 1
              num_followers = parsed_obj['author']['followers']
              author_name = parsed_obj['author']['name']
              num_retweets = parsed_obj['metrics']['citations']['total']
              total_tweets = 1 
              mentions = len(parsed_obj['tweet']['entities']['user_mentions'])
              urls = len(parsed_obj['tweet']['entities']['urls'])
              favorite_count = parsed_obj['tweet']['favorite_count']
              impressions_num = parsed_obj['metrics']['impressions']
              author_name = parsed_obj['author']['name']

              if(time_inst_hr in num_tweets ):
                  num_tweets[time_inst_hr] += total_tweets
              else:
                 num_tweets[time_inst_hr] = total_tweets  

              if(time_inst_hr in total_retweets ):
                  total_retweets[time_inst_hr] += num_retweets
              else:
                 total_retweets[time_inst_hr] = num_retweets 

              if(time_inst_hr in sum_followers):
                  sum_followers[time_inst_hr] += num_followers
              else:
                 sum_followers[time_inst_hr] = num_followers     

              if(time_inst_hr in max_followers):
                if num_followers > max_followers[time_inst_hr]:
                  max_followers[time_inst_hr] = num_followers
              else:
                 max_followers[time_inst_hr] = num_followers  

              if(time_inst_hr in user_mentions):
                  user_mentions[time_inst_hr] += mentions
              else:
                 user_mentions[time_inst_hr] = mentions

              if(time_inst_hr in url_count):
                  url_count[time_inst_hr] += urls
              else:
                 url_count[time_inst_hr] = urls   

              if(time_inst_hr in fav_count):
                  fav_count[time_inst_hr] += favorite_count
              else:
                 fav_count[time_inst_hr] = favorite_count   

              if(time_inst_hr in impressions_count):
                  impressions_count[time_inst_hr] += impressions_num
              else:
                 impressions_count[time_inst_hr] = impressions_num 

              if(time_inst_hr in author_names):
                  author_names[time_inst_hr].append(author_name)
              else:
                 author_names[time_inst_hr] = [author_name]    
          
              # reset the values to 0 for next iteration
              num_followers = 0
              num_retweets  = 0
              total_tweets  = 0
          else: 
              #print("skipped citation_time",citation_time)
              num_followers = 0
              num_retweets  = 0
              total_tweets  = 0
            
          #if count == 100:
          #  break
# print(author_names)
X_data = [] 
Y_data = []
#just to get the keys    
for key in sorted (num_tweets.keys()):
    #print(key)
    #value = int(key)
    time_of_day = int(int(datetime.datetime.fromtimestamp(key).strftime('%H')))
    #authors_list = author_names[key]
    unique_authors_count = len(set(author_names[key]))
    day_value = int(datetime.datetime.fromtimestamp(key).strftime('%w'))
    #print("authrs list and count:",authors_list,unique_authors_count)
    X[key] = [num_tweets[key],total_retweets[key],sum_followers[key],max_followers[key],user_mentions[key],url_count[key],fav_count[key],impressions_count[key],unique_authors_count]
    # Num_tweets, total_retweets, sum of followers , max_followers , time of tweet , user mentions, url_count,fav_count,impressions, unique_authors_count
    X_data.append([num_tweets[key],total_retweets[key],sum_followers[key],max_followers[key],time_of_day,user_mentions[key],url_count[key],fav_count[key],impressions_count[key],unique_authors_count,day_value])
    Y_data.append([num_tweets[key]])
#print(X_data)
#print(Y_data)
Y = collections.deque(Y_data)
#print(Y)
Y.rotate(-1)
#print(Y)
Y = list(Y)
#print(Y)
      
      
Y = np.ravel(Y)

NameError: ignored

In [0]:
from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
'max_depth': [10, 20, 40, 60, 80, 100, 200, None],
'max_features': ['auto', 'sqrt'],
'min_samples_leaf': [1, 2, 4],
'min_samples_split': [2, 5, 10],
'n_estimators': [200, 400, 600, 800, 1000,
1200, 1400, 1600, 1800, 2000]
}
# Create a based model
from sklearn.ensemble import GradientBoostingRegressor

gb  = GradientBoostingRegressor(random_state=0)


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = gb, param_grid = param_grid, 
                         cv = KFold(5, shuffle=True), scoring='neg_mean_squared_error')

grid_search.fit(X_data, Y)
grid_search.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'n_estimators': 1400}

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size = 0.2, random_state = 0)

#def evaluate(model, test_features, test_labels):
    #predictions = model.predict(test_features)
    #errors = abs(predictions - test_labels)
    #mape = 100 * np.mean(errors / test_labels)
    #accuracy = 100 - mape
    #print('Model Performance')
    #print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    #return accuracy

#from sklearn.metrics import accuracy_score
best_grid = grid_search.best_estimator_
print(best_grid)
#grid_accuracy = evaluate(best_grid, X_test, y_test)


gb =  GradientBoostingRegressor(random_state=0, n_estimators=1400, max_depth=10, max_features='sqrt', 
                                  min_samples_leaf=2, min_samples_split=10)
gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)

from sklearn import metrics

print("Mean Square Error", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean-Square Error" , np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# print("Accuracy ", metrics.accuracy_score(Y, Y_pred))

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=10,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=2, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=1400,
                          n_iter_no_change=None, presort='auto', random_state=0,
                          subsample=1.0, tol=0.0001, validation_fraction=0.1,
                          verbose=0, warm_start=False)


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Square Error 1633763959.6374383
Root Mean-Square Error 40419.84611100639
